======================================

In [ ]:
# ============================================================================
# EGYPTIAN ID OCR - NUMPY FIX FOR GOOGLE COLAB
# ============================================================================
# This fixes the NumPy 2.x compatibility issue
# Run this cell, then RESTART RUNTIME

import sys
import subprocess

print("🚀 Installing Egyptian ID OCR - NumPy Fixed Version")
print("=" * 70)

# ─── CRITICAL FIX: Downgrade NumPy FIRST ────────────────────────────────────
print("\n🔧 Step 1/4: Fixing NumPy compatibility issue...")
print("   (Downgrading NumPy 2.4.1 → NumPy 1.26.4)")

subprocess.run([sys.executable, "-m", "pip", "install", "-q",
                "numpy==1.26.4", "--force-reinstall"],
               stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
print("✅ NumPy 1.26.4 installed")

# ─── STEP 2: Install core packages ─────────────────────────────────────────
print("\n📦 Step 2/4: Installing core packages...")

core_packages = [
    "ultralytics",
    "roboflow",
    "arabic-reshaper",
    "python-bidi",
    "openpyxl",
]

for pkg in core_packages:
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", pkg],
                   stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    print(f"✅ {pkg}")

# ─── STEP 3: Install PaddleOCR ─────────────────────────────────────────────
print("\n📦 Step 3/4: Installing PaddleOCR...")

# Install compatible PaddlePaddle version
subprocess.run([sys.executable, "-m", "pip", "install", "-q",
                "paddlepaddle-gpu==2.6.1"],
               stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
print("✅ paddlepaddle-gpu 2.6.1")

subprocess.run([sys.executable, "-m", "pip", "install", "-q", "paddleocr==2.7.3"],
               stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
print("✅ paddleocr 2.7.3")

# ─── STEP 4: Pin NumPy to prevent auto-upgrade ─────────────────────────────
print("\n📦 Step 4/4: Locking NumPy version...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q",
                "numpy==1.26.4", "--no-deps"],
               stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
print("✅ NumPy locked at 1.26.4")

print("\n" + "=" * 70)
print("✅ INSTALLATION COMPLETE!")
print("=" * 70)
print("\n⚠️  CRITICAL: You MUST restart runtime now!")
print("\n📝 Next steps:")
print("   1. Runtime → Restart runtime")
print("   2. Run the test cell to verify")
print("   3. Do NOT run any other cells until after restart")
print("\n💡 After restart, NumPy will stay at 1.26.4 and everything will work")
print("=" * 70)

# New Section

In [ ]:
import numpy as np
print(np.__version__)  # يجب أن يكون 1.26.4

from paddleocr import PaddleOCR
ocr = PaddleOCR(lang='ar', use_angle_cls=True, use_gpu=False, show_log=False)
print("✅ PaddleOCR ready")


In [ ]:
import os, cv2, numpy as np, pandas as pd, matplotlib.pyplot as plt
from PIL import Image
import json, re, yaml
from datetime import datetime
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import arabic_reshaper
from bidi.algorithm import get_display

print("✅ Imports done!")

In [ ]:
# ─── CELL 3: GOOGLE DRIVE ───────────────────────────────────────────────────

from google.colab import drive
drive.mount('/content/drive')

PROJECT_DIR = '/content/drive/MyDrive/Egyptian_ID_Final'
os.makedirs(PROJECT_DIR, exist_ok=True)

for d in ['models', 'outputs', 'results']:
    os.makedirs(f'{PROJECT_DIR}/{d}', exist_ok=True)

print(f"✅ Project: {PROJECT_DIR}")

In [ ]:

# ─── CELL 4: DOWNLOAD DATASET ───────────────────────────────────────────────

from roboflow import Roboflow

# ⚠️ REPLACE WITH YOUR CREDENTIALS
API_KEY = "SNKUr0sNWb9nWYv1KO5R"
WORKSPACE = "mywork-f0ue6"
PROJECT = "ids-z3sti"
VERSION = 4

rf = Roboflow(api_key=API_KEY)
project = rf.workspace(WORKSPACE).project(PROJECT)
dataset = project.version(VERSION).download("yolov8")

with open(f"{dataset.location}/data.yaml") as f:
    config = yaml.safe_load(f)

print(f"✅ Dataset: {dataset.location}")
print(f"📊 Classes: {config['names']}")

In [ ]:

# ─── CELL 5: TRAIN MODEL (SKIP IF ALREADY TRAINED) ─────────────────────────

from ultralytics import YOLO

MODEL_PATH = f'{PROJECT_DIR}/models/best.pt'

if os.path.exists(MODEL_PATH):
    print(f"✅ Using existing model")
    model = YOLO(MODEL_PATH)
else:
    print("🚀 Training YOLOv8...")
    model = YOLO('yolov8n.pt')

    model.train(
        data=f'{dataset.location}/data.yaml',
        epochs=150,
        imgsz=640,
        batch=16,
        device=0,
        patience=20,
        project=f'{PROJECT_DIR}/models',
        name='detector',
        augment=True,
        hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
        degrees=5, translate=0.1, scale=0.2
    )

    import shutil
    best = f'{PROJECT_DIR}/models/detector/weights/best.pt'
    shutil.copy(best, MODEL_PATH)
    model = YOLO(MODEL_PATH)

print("✅ Model ready!")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ─── CELL 6: EVALUATE ───────────────────────────────────────────────────────

metrics = model.val()

print(f"\n📊 mAP@50: {metrics.box.map50:.3f}")
print(f"📊 Precision: {metrics.box.mp:.3f}")
print(f"📊 Recall: {metrics.box.mr:.3f}\n")

for i, name in enumerate(config['names']):
    if i < len(metrics.box.ap50):
        print(f"  {name:15s}: {metrics.box.ap50[i]:.1%}")


In [ ]:
class Preprocessor:
    @staticmethod
    def enhance(img, target_size=(1024,1024)):
        # Resize for consistent input
        img = cv2.resize(img, target_size)

        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape)==3 else img

        # Denoise & sharpen
        gray = cv2.GaussianBlur(gray, (3,3), 0)
        kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
        gray = cv2.filter2D(gray, -1, kernel)

        # CLAHE (adaptive histogram equalization)
        clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8,8))
        gray = clahe.apply(gray)

        # Adaptive Threshold for better OCR
        return cv2.adaptiveThreshold(gray, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 199, 45)


In [ ]:
# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="SNKUr0sNWb9nWYv1KO5R")
# project = rf.workspace("mywork-f0ue6").project("ids-z3sti")
# version = project.version(4)
# dataset = version.download("yolov8")



In [ ]:
import numpy as np
print("NumPy:", np.__version__)

from paddleocr import PaddleOCR
ocr = PaddleOCR(lang='ar', use_angle_cls=True, use_gpu=False, show_log=False)

print("✅ PaddleOCR READY")


In [ ]:

import os
import cv2
import numpy as np
import re
from datetime import datetime
import arabic_reshaper
from bidi.algorithm import get_display
from pathlib import Path

In [ ]:

# ============================================================
# 1️⃣ INSTALL & IMPORT PACKAGES
# ============================================================

!pip install -q paddleocr paddlepaddle numpy==1.26.4 opencv-python matplotlib arabic-reshaper python-bidi easyocr
!apt-get install -q tesseract-ocr tesseract-ocr-ara

import cv2, numpy as np, os, re
from paddleocr import PaddleOCR
import arabic_reshaper
from bidi.algorithm import get_display
from datetime import datetime
from pathlib import Path
import matplotlib.pyplot as plt

print("✅ Packages ready")

In [ ]:
# ============================================================
# 2️⃣ ENHANCED PREPROCESSING CLASS
# ============================================================

class Preprocessor:
    @staticmethod
    def enhance(img, aggressive=False):
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape)==3 else img.copy()
        h, w = gray.shape[:2]

        # Resize small images
        if max(h, w) < 200:
            scale = 4 if aggressive else 3
            gray = cv2.resize(gray, (w*scale, h*scale), interpolation=cv2.INTER_CUBIC)

        # Denoise
        gray = cv2.fastNlMeansDenoising(gray, None, 10, 7, 21)

        # Sharpen
        kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
        gray = cv2.filter2D(gray, -1, kernel)

        # CLAHE
        clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(16,16))
        gray = clahe.apply(gray)

        # Adaptive Threshold for ID numbers
        if aggressive:
            gray = cv2.adaptiveThreshold(gray, 255,
                                         cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                         cv2.THRESH_BINARY, 11, 2)

        # Add border
        gray = cv2.copyMakeBorder(gray, 20, 20, 20, 20,
                                  cv2.BORDER_CONSTANT, value=255)
        return gray

    @staticmethod
    def correct_rotation(img):
        """Detect edges and rotate to correct skew"""
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


In [ ]:
import easyocr
import cv2
import numpy as np

class ProductionOCR:
    def __init__(self):
        # تحميل EasyOCR للغة العربية والأرقام
        self.reader = easyocr.Reader(['ar', 'en'], gpu=True) # تأكد إن الـ GPU شغال
        print("✅ EasyOCR Engine Loaded (Final Fix)")

    def extract(self, img_crop, field_type):
        try:
            # معالجة الصورة قبل البعث لـ OCR
            gray = cv2.cvtColor(img_crop, cv2.COLOR_BGR2GRAY)
            # تكبير بسيط لتحسين القراءة
            gray = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

            # في حالة الرقم القومي، نستخدم Threshold لزيادة التباين
            if field_type == 'id_number':
                _, gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            # تنفيذ الـ OCR
            results = self.reader.readtext(gray)

            if not results:
                return {'text': '', 'conf': 0.0}

            # تجميع النص
            texts = [res[1] for res in results]
            full_text = " ".join(texts)
            conf = np.mean([res[2] for res in results])

            # تنظيف النص باستخدام الـ Clean اللي عندك
            clean_text = self._clean(full_text, field_type)

            return {
                'text': clean_text,
                'conf': float(conf),
                'source': 'easyocr_v2'
            }
        except Exception as e:
            return {'text': f'Error: {str(e)}', 'conf': 0.0}

    def _clean(self, text, field_type):
        # تنظيف بسيط وسريع
        if field_type == 'id_number':
            digits = re.findall(r'\d+', text.replace(' ', ''))
            res = "".join(digits)
            return res[-14:] if len(res) >= 14 else res

        # للأسماء والعناوين، شيل الحروف اللاتينية والرموز
        text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
        return text.strip()

In [ ]:
# ─── CELL 9: EGYPTIAN ID LOGIC ──────────────────────────────────────────────

class IDLogic:
    GOVS = {'01':'القاهرة','02':'الإسكندرية','03':'بورسعيد','04':'السويس',
            '11':'دمياط','12':'الدقهلية','13':'الشرقية','14':'القليوبية',
            '15':'كفر الشيخ','16':'الغربية','17':'المنوفية','18':'البحيرة',
            '19':'الإسماعيلية','21':'الجيزة','22':'بني سويف','23':'الفيوم',
            '24':'المنيا','25':'أسيوط','26':'سوهاج','27':'قنا','28':'أسوان',
            '29':'الأقصر','31':'البحر الأحمر','32':'الوادي الجديد','33':'مطروح',
            '34':'شمال سيناء','35':'جنوب سيناء'}

    @staticmethod
    def parse(id_num):
        id_num = re.sub(r'\D', '', id_num)
        if len(id_num) != 14:
            return {'valid': False}

        try:
            century = int(id_num[0])
            year = (1900 if century==2 else 2000) + int(id_num[1:3])
            month = int(id_num[3:5])
            day = int(id_num[5:7])
            gov_code = id_num[7:9]
            gender = 'ذكر' if int(id_num[12])%2==1 else 'أنثى'
            governorate = IDLogic.GOVS.get(gov_code, f'Unknown({gov_code})')
            age = datetime.now().year - year

            return {
                'valid': True,
                'birth_date': f"{day:02d}/{month:02d}/{year}",
                'gender': gender,
                'governorate': governorate,
                'age': age
            }
        except:
            return {'valid': False}

In [ ]:
class Pipeline:
    """Complete Egyptian ID Processing Pipeline - Optimized Version"""

    def __init__(self, model_path):
        from ultralytics import YOLO

        # 1. تحميل موديل YOLO بتاعك
        self.model = YOLO(model_path)

        # 2. تحميل محرك الـ OCR الجديد (EasyOCR/Paddle)
        # مبيحتاجش Arguments دلوقتي لأنه متظبط جوه الكلاس بتاعه
        self.ocr = ProductionOCR()

        # 3. خريطة الكلاسات (تأكد إن الترتيب ده هو اللي في الـ YAML عندك)
        self.field_map = {
            0: 'address',
            1: 'birth_date',
            2: 'country',
            3: 'full_name',
            4: 'id_number'
        }

        print("✅ New Pipeline ready without Google Vision conflicts!")

    def process(self, img_path, visualize=True):
        start = datetime.now()

        # Load image
        img = cv2.imread(img_path) if isinstance(img_path, str) else img_path
        if img is None:
            return {'success': False, 'error': 'Failed to load image'}

        # Detection
        results = self.model(img, conf=0.25, verbose=False)

        data = {}
        for result in results:
            for box in result.boxes:
                cid = int(box.cls[0])
                det_conf = float(box.conf[0])
                field = self.field_map.get(cid)

                if not field: continue

                # Crop
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                # إضافة Padding بسيط عشان الحروف اللي على الأطراف
                h, w = img.shape[:2]
                pad = 15
                crop = img[max(0,y1-pad):min(h,y2+pad), max(0,x1-pad):min(w,x2+pad)]

                # OCR Extraction
                ocr_result = self.ocr.extract(crop, field)

                data[field] = {
                    'text': ocr_result['text'],
                    'confidence': min(det_conf, ocr_result['conf']),
                    'bbox': [x1, y1, x2, y2],
                    'source': ocr_result.get('source', 'new_engine')
                }

        # Logic للرقم القومي (فك الشفرة)
        id_info = None
        if 'id_number' in data and len(data['id_number']['text']) >= 14:
            id_info = IDLogic.parse(data['id_number']['text'])
            if id_info.get('valid'):
                # تعبئة البيانات التلقائية من الرقم القومي
                data['birth_date_from_id'] = {'text': id_info['birth_date'], 'confidence': 1.0}
                data['governorate'] = {'text': id_info['governorate'], 'confidence': 1.0}
                data['gender'] = {'text': id_info['gender'], 'confidence': 1.0}

        return {
            'success': True,
            'time': (datetime.now() - start).total_seconds(),
            'fields': data,
            'id_logic': id_info
        }

In [ ]:
# ─── CELL 5: DISPLAY RESULTS ───────────────────────────────────────────────

def show_result(result):
    """Display extraction results"""

    print("\n" + "="*80)
    print("🆔 نتائج استخراج بيانات البطاقة")
    print("="*80)

    if not result['success']:
        print(f"❌ خطأ: {result.get('error')}")
        return

    print(f"\n⏱️  الوقت: {result['time']:.2f} ثانية\n")

    print("📋 البيانات المستخرجة:")
    print("-" * 80)

    fields_ar = {
        'full_name': 'الاسم',
        'id_number': 'الرقم القومي',
        'birth_date': 'تاريخ الميلاد',
        'address': 'العنوان',
        'country': 'الجنسية'
    }

    for field in ['full_name', 'id_number', 'birth_date', 'address', 'country']:
        if field in result['fields']:
            d = result['fields'][field]
            ar_name = fields_ar[field]
            print(f"  {ar_name:15s}: {d['text']}")
            print(f"  {'':15s}  الثقة: {d['confidence']:.1%} | المصدر: {d.get('source', 'N/A')}")

    if result.get('id_logic') and result['id_logic'].get('valid'):
        print("\n🧠 معلومات مستخرجة من الرقم القومي:")
        print("-" * 80)
        info = result['id_logic']
        print(f"  تاريخ الميلاد: {info['birth_date']}")
        print(f"  العمر: {info['age']} سنة")
        print(f"  النوع: {info['gender']}")
        print(f"  المحافظة: {info['governorate']}")

    if result.get('viz_path'):
        print(f"\n💾 الصورة المعالجة: {result['viz_path']}")

        try:
            import matplotlib.pyplot as plt
            img = cv2.imread(result['viz_path'])
            if img is not None:
                plt.figure(figsize=(12, 8))
                plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                plt.title('نتائج الكشف')
                plt.axis('off')
                plt.show()
        except:
            pass

    print("="*80)

In [ ]:
def upload_and_process():
    """رفع ومعالجة البطاقة"""
    from google.colab import files

    print("📤 ارفع صورة البطاقة:")
    uploaded = files.upload()

    if not uploaded:
        return None

    fname = list(uploaded.keys())[0]
    temp = f'/tmp/{fname}'

    with open(temp, 'wb') as f:
        f.write(uploaded[fname])

    print(f"\n✅ جاري المعالجة: {fname}\n")

    result = pipe.process(temp)
    show_result(result)

    return result


print("\n🚀 جاهز! شغلي: upload_and_process()")

In [ ]:
# هنا بنبعت الـ MODEL_PATH بس، ومفيش google_credentials خالص
pipe = Pipeline(MODEL_PATH)

# ابدأ الرفع والمعالجة
upload_and_process()